In [1]:
import os
import numpy as np
import numpy.random  as rd
import scipy as sp
from scipy import stats as st
from collections import Counter
import pandas as pd
import itertools

import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
import matplotlib.animation as ani
import random
plt.rcParams["patch.force_edgecolor"] = True
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]
from IPython.display import display
%matplotlib inline

In [2]:
org_data = pd.read_csv("../data/kddcup.data.corrected")
header_data = pd.read_csv("../data/header.csv")
display(header_data)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate


In [3]:
header = list(header_data.columns)
header.append("label")
org_data.columns = header

In [4]:
display(org_data.head())
display(org_data.tail())
print(org_data.shape)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,238,1282,0,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
4898425,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898426,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1098,0,0,0,0,...,255,1.0,0.0,0.14,0.05,0.0,0.01,0.0,0.0,normal.


(4898430, 42)


In [5]:
df = org_data.copy()
display(df.head())

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,238,1282,0,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.


In [6]:
SF = df.loc[df.logged_in > 0]
SF10 = SF.sample(n=100000)
SF10.shape

(100000, 42)

In [7]:
dataset = SF10[["duration", "src_bytes", "dst_bytes", "label"]]
display(dataset.head())
display(dataset.tail())
display(dataset.describe())

for i in dataset.columns[:-1]:
    dataset[i] = np.log(dataset[i] + 0.1)
    
display(dataset.head())
display(dataset.tail())
display(dataset.describe())

,duration,src_bytes,dst_bytes,label
1360670,0,238,8515,normal.
4790025,0,214,812,normal.
281968,0,293,5415,normal.
236435,0,286,4708,normal.
755599,0,641,0,normal.


,duration,src_bytes,dst_bytes,label
4885731,0,202,659,normal.
132485,0,288,301,normal.
814356,0,232,438,normal.
5642,0,289,12640,normal.
767308,0,211,4880,normal.


,duration,src_bytes,dst_bytes
count,100000.000000,1.000000e+05,1.000000e+05
mean,12.271970,2.373781e+03,4.389843e+03
std,369.117429,8.821694e+04,3.596507e+04
min,0.000000,0.000000e+00,0.000000e+00
25%,0.000000,2.220000e+02,3.500000e+02
50%,0.000000,2.810000e+02,1.191000e+03
75%,0.000000,3.320000e+02,3.222000e+03
max,28819.000000,1.881556e+07,3.916592e+06


/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,duration,src_bytes,dst_bytes,label
1360670,-2.302585,5.472691,9.049596,normal.
4790025,-2.302585,5.366443,6.699623,normal.
281968,-2.302585,5.680514,8.596947,normal.
236435,-2.302585,5.656341,8.457040,normal.
755599,-2.302585,6.463185,-2.302585,normal.


,duration,src_bytes,dst_bytes,label
4885731,-2.302585,5.308763,6.490875,normal.
132485,-2.302585,5.663308,5.707442,normal.
814356,-2.302585,5.447168,6.082447,normal.
5642,-2.302585,5.666773,9.444630,normal.
767308,-2.302585,5.352332,8.492921,normal.


,duration,src_bytes,dst_bytes
count,100000.000000,100000.000000,100000.000000
mean,-1.991415,5.847608,6.822265
std,1.131160,0.911021,2.393200
min,-2.302585,-2.302585,-2.302585
25%,-2.302585,5.403128,5.858219
50%,-2.302585,5.638710,7.082633
75%,-2.302585,5.805436,8.077789
max,10.268794,16.750195,15.180732


In [8]:
def d_h_Gauss(mu, sigma, mu_prev, sigma_prev):
    '''
    t時点とt-1時点におけるガウス分布間のヘリンジャー距離
    '''
    mu = mu[:, np.newaxis]
    mu_prev = mu_prev[:, np.newaxis]
    #第2項
    m = 2 * np.linalg.det((np.linalg.inv(sigma) + np.linalg.inv(sigma_prev)) / 2) ** (-1/2) #分子
    d = (np.linalg.det(sigma) ** (1/4)) * (np.linalg.det(sigma_prev) ** (1/4)) #分母
    A = m / d
    
    #第3項 1番目exp
    B_0 = (np.dot(np.linalg.inv(sigma), mu) + np.dot(np.linalg.inv(sigma_prev), mu_prev)).T #要素1
    B_1 = np.linalg.inv(np.linalg.inv(sigma) + np.linalg.inv(sigma_prev)) #要素2
    B_2 = np.dot(np.linalg.inv(sigma), mu) + np.dot(np.linalg.inv(sigma_prev), mu_prev) #要素3
    B = np.exp((1/2) * np.dot(np.dot(B_0, B_1), B_2))
    
    #第3項 2番目exp
    C_0 = np.dot(np.dot(mu.T, np.linalg.inv(sigma)), mu) + np.dot(np.dot(mu_prev.T, np.linalg.inv(sigma_prev)), mu_prev) #要素
    C = np.exp(-(1/2) * C_0)
    
    #ヘリンジャー距離の計算
    d_h = 2 - A * B * C
    
    return d_h


def d_h_GMM(pi, pi_prev, mu, sigma, mu_prev, sigma_prev, k):
    '''
    t時点とt-1時点における混合ガウス分布のヘリンジャー距離
    '''
    d_h = 0
    for i in range(k):
        d_h_G = d_h_Gauss(mu[i], sigma[i], mu_prev[i], sigma_prev[i])
        d_h += (np.sqrt(pi[i]) - np.sqrt(pi_prev[i])) ** 2 + (pi[i] + pi_prev[i])/2 * d_h_G
    
    return d_h


def calc_hellinger_score(p, p_prev, sdem, t, r, k, M):
    '''
    ヘリンジャースコアを計算する関数
    '''
    S_H = 0
    for m in range(M):
        d_h = d_h_GMM(sdem[m].pi[t], sdem[m].pi[t-1] ,sdem[m].mu[t], sdem[m].sigma[t], 
                      sdem[m].mu[t-1], sdem[m].sigma[t-1], k)
        tmp = np.sqrt(p[m] * p_prev[m]) #√(p_t(x)*p_t_1(x))      
        S_H += tmp * (d_h - 2)
    S_H += 2
    S_H /= r**2
    
    return S_H

In [9]:
os.chdir("..")
home_dir = os.getcwd()
os.chdir("src")
from sdle import SDLE
from sdem import SDEM
os.chdir(home_dir)

In [10]:
x = np.array([0 for i in range(dataset.shape[0])])
y = np.array(dataset.iloc[:, :-1])
K = 2
D = 3

In [11]:
#パラメータの設定
#SDLE
A = list(itertools.product(set(x)))
r = 0.0002 #忘却パラメータ
beta = 1 #正の定数


#SDEM
alpha = 2.0 #(1.0~2.0)
r = r
k = K #潜在変数の数
d = D #次元
# calc_helinger_score
r_h = 0.0003

In [12]:
# オブジェクトの初期化
sdle = SDLE(r, beta, A)
sdem = [SDEM(r, alpha, k, d) for i in range(len(A))]

#結果保存用
p = np.zeros((1,len(A))) #同時確率
S_H = [] #ヘリンジャースコア
S_L = [] #シャノン情報量(対数損失)

In [13]:
%%time
t = 1
while t <= len(x):
    #SDLE
    sdle.update(x[t-1])
    #p に一行追加
    p_new = np.zeros((1,len(A)))
    p = np.concatenate([p, p_new])
    
    #SDEM: 各セルに対応する混合ガウス分布のパラメータと確率を推定
    for m, A_m in enumerate(sdle.A):
        if m == int(sdle.flag):
            sdem[m].update(y[t-1])
            p_y_x = np.dot(sdem[m].prob[t], sdem[m].pi[t])
            p[t, m] = p_y_x * sdle.prob[t, int(sdle.flag)] #t時点における同時確率:p_t(x,y)
        else:
            sdem[m].skip()
            p[t, m] = p[t-1, m]
    
    #スコア計算(シャノン情報量)
    p_x_prev = sdle.prob[t-1, sdle.flag]
    pi_prev = sdem[int(sdle.flag)].pi[t-1]
    mu_prev = sdem[int(sdle.flag)].mu[t-1]
    sigma_prev = sdem[int(sdle.flag)].sigma[t-1]
    p_Gauss_prev = sdem[int(sdle.flag)].calc_prob(y[t-1], mu_prev, sigma_prev)
    p_y_x_prev = np.dot(pi_prev, p_Gauss_prev)
    p_prev_params = p_x_prev * p_y_x_prev #t-1時点のパラメータによる同時確率:P(x,y)
    s_l = -np.log(p_prev_params) #シャノン情報量を計算
    S_L.append(s_l)
    
    #ヘリンジャースコア
    s_h = calc_hellinger_score(sdle.prob[t], sdle.prob[t-1], sdem, t, r_h, k, sdle.M)
    S_H.append(s_h)
    
    t += 1
    if t % 1000 == 0:
        print(t)

/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:28: RuntimeWarning: divide by zero encountered in log


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000


/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in multiply
/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:16: RuntimeWarning: overflow encountered in exp
/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:23: RuntimeWarning: invalid value encountered in multiply


45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
CPU times: user 28min 58s, sys: 9min 21s, total: 38min 19s
Wall time: 13min 17s


In [14]:
dataset["S_L"] = np.array(S_L)
dataset["S_H"] = np.array(S_H)[:, 0, 0]
dataset.describe()

/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,duration,src_bytes,dst_bytes,S_L,S_H
count,100000.000000,100000.000000,100000.000000,1.000000e+05,4.472300e+04
mean,-1.991415,5.847608,6.822265,inf,-inf
std,1.131160,0.911021,2.393200,NaN,NaN
min,-2.302585,-2.302585,-2.302585,-2.219157e+00,-inf
25%,-2.302585,5.403128,5.858219,-9.102283e-01,2.268140e-01
50%,-2.302585,5.638710,7.082633,1.737829e+00,3.180052e-01
75%,-2.302585,5.805436,8.077789,4.890707e+00,7.639055e-01
max,10.268794,16.750195,15.180732,inf,2.222222e+07


In [15]:
top10_S_L = dataset.sort(columns="S_L", ascending=False)[:10000]
ratio_s_l = top10_S_L.loc[top10_S_L.label != "normal."].shape[0]/dataset.loc[dataset.label != "normal."].shape[0]
top10_S_H = dataset.sort(columns="S_H", ascending=False)[:10000]
ratio_s_h = top10_S_H.loc[top10_S_H.label != "normal."].shape[0]/dataset.loc[dataset.label != "normal."].shape[0]
print(ratio_s_l, ratio_s_h)

0.9572072072072072 0.46396396396396394


/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [16]:
top20_S_L = dataset.sort(columns="S_L", ascending=False)[:20000]
ratio_s_l = top20_S_L.loc[top20_S_L.label != "normal."].shape[0]/dataset.loc[dataset.label != "normal."].shape[0]
top20_S_H = dataset.sort(columns="S_H", ascending=False)[:20000]
ratio_s_h = top20_S_H.loc[top20_S_H.label != "normal."].shape[0]/dataset.loc[dataset.label != "normal."].shape[0]
print(ratio_s_l, ratio_s_h)

/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


0.990990990990991 0.46621621621621623
